# Import related pacakages

In [1]:
import cobra
import sys
sys.path.append(r'./code/')
from cobrapy_ec_model_function import *

# Inputing files

In [2]:
# The genome-scale metabolic model for constructing the enzyme-constrained model
model_name = './data/iML1515.xml' 

# Reaction-kcat file.#s-1
# eg. AADDGT,49389.2889
#reaction_kcat_file = "./data/reaction_autopacmen_ori_noadj.csv"
#reaction_kcat_file = "./data/reaction_kappori.csv"
reaction_kcat_file = "./data/reaction_autopacmen_ori_adj.csv"

# Gene-abundance file. 
# eg. b0789,1.1
gene_abundance_file = "./data/gene_abundance.csv"

# Gene-molecular_weight file. 
# eg. b0001,thrL,2.13846
gene_molecular_weight_file = "./data/gene_molecular_weight.csv"


# Step1: Preprocessing of model

The reversible reactions in the GEM model are divided into two irreversible reactions and splited isoenzyme. The input is iML1515 with 2712 reactions. The output is a model with 3375 irreversible reactions.

In [3]:
model = cobra.io.read_sbml_model(model_name)
convert_to_irreversible(model)
#split isoenzyme
model = isoenzyme_split(model)
model

Name,iML1515
Memory address,0x01e6b73707b8
Number of metabolites,1877
Number of reactions,5920
Number of groups,38
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


# Step2: Retrieving enzyme kinetics and proteomics data

The inputs are GEM model. The outputs are 'genes' and 'gpr_relationship' data in the iML1515.

In [4]:
[genes,gpr_relationship] = get_genes_and_gpr(model)

Get the molecular weight of the enzyme (MW) according to the file of all_reaction_GPR.csv, which obtained from the previous step (gpr_relationship, ./analysis/all_reaction_GPR.csv). We need to manually correct the error of the gene_reaction_rule of a small amount of reactions in iML1515 (See Supplementary Table S1 for details), and also need to manually get the subunit of each protein from EcoCyc.

In [5]:
reaction_gene_subunit_MW = pd.read_csv("./data/reaction_gene_subunit_MW_old.csv", index_col=0)
reaction_gene_subunit_MW_split(reaction_gene_subunit_MW)

In [6]:
# reaction-gene-subunit-MW file. 
# eg. ALATA_D2,D-alanine transaminase,b2551 or b0870,45.31659 or 36.49471 ,2 or 4 
reaction_gene_subunit_MW = "./data/reaction_gene_subunit_MW.csv"
reaction_mw = calculate_reaction_mw(reaction_gene_subunit_MW)

Calculate kcat/MW. The inputs are 'reaction_kcat' and 'reaction_MW' data for calculating the kcat/MW (When the reaction is catalyzed by several isozymes, the maximum is retained).

In [7]:
save_file="./analysis/reaction_kcat_mw.csv"
reaction_kcat_mw = calculate_reaction_kcat_mw(reaction_kcat_file, reaction_mw, save_file)

Calculate f. The input is 'genes' data, 'gene_abundance.csv' and 'gene_molecular_weight.csv'.

In [8]:
f = calculate_f(genes, gene_abundance_file, gene_molecular_weight_file)
f

0.4059986079578236

# Step3: Save enzyme concentration constraint model as json file.

In [9]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kcat_mw.csv"
json_output_file="./model/iML1515_irr_enz_constraint_kapp.json"
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
sigma = 0.5 
#sigma = 1 #kapp data sigma is 1
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model_split_isoenzyme(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [10]:
json_model_path = "./model/iML1515_irr_enz_constraint_kapp.json"
fluxes_outfile = './analysis/ECMpy_kappori_solution_df_pfba.csv'
reaction_kcat_mw_file="./analysis/reaction_kcat_mw.csv"
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile,reaction_kcat_mw_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

norm_model = cobra.io.json.load_json_model(json_model_path)
norm_model_pfba_solution = cobra.flux_analysis.pfba(norm_model)
norm_model_pfba_solution_df = norm_model_pfba_solution.to_frame()
norm_model_pfba_solution_df.to_csv('./analysis/Orimodel_solution_df_pfba.csv')
print(norm_model_pfba_solution_df.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.37468813231840686
0.8769972144269685


# Step4: Calibration parameters

Compare with C13 data

In [11]:
import re
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = []
for eachreaction in enz_model_pfba_solution_select.index:
    if re.search('_num',eachreaction):
        enz_model_pfba_solution_select_id.append(eachreaction.split('_num')[0])
    else:
        enz_model_pfba_solution_select_id.append(eachreaction)

c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['TKT2', 'TPI', 'SUCDi', 'GND', 'SUCOAS_reverse', 'AKGDH', 'RPE', 'TALA', 'PFK', 'ME2', 'TKT1', 'ICL', 'FBA']


Select calibration reaction accrording biomass diff

In [12]:
reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"
json_model_path = "./model/iML1515_irr_enz_constraint_kapp.json"
enzyme_amount = upperbound
percentage = 0.1#10%
reaction_biomass_outfile = './analysis/reaction_biomass.csv'
select_percentage = 0.8
df_reaction_select = select_calibration_reaction_by_biomass(reaction_kcat_mw_file, json_model_path, enzyme_amount, percentage, reaction_biomass_outfile, select_percentage)
df_reaction_select

,biomass_enz,biomass_orimodel,biomass_diff,biomass_diff_ratio
KARA1_reverse,0.310947,0.876997,0.56605,0.645441


In [13]:
need_change_reaction=list(df_reaction_select.index)
reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change.csv"
kcat_data_colect_file="./analysis/reaction_max_df.csv"
change_reaction_list_round1=change_reaction_kcat_by_database_g(need_change_reaction,kcat_data_colect_file,reaction_kcat_mw_file,reaction_kapp_change_file)
print(change_reaction_list_round1)

['KARA1_reverse']


In [14]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kapp_change.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_round1.json"
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
sigma = 0.5 
#sigma = 1#kcapp数据，饱和度为1
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model_split_isoenzyme(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [15]:
json_model_path = "./model/iML1515_irr_enz_constraint_adj_round1.json"
fluxes_outfile= './analysis/ECMpy_kapp_adj_round1_solution_df_pfba.csv'
reaction_kcat_mw_file="./analysis/reaction_kapp_change.csv"
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile,reaction_kcat_mw_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.4243615507210177


In [16]:
import re
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = []
for eachreaction in enz_model_pfba_solution_select.index:
    if re.search('_num',eachreaction):
        enz_model_pfba_solution_select_id.append(eachreaction.split('_num')[0])
    else:
        enz_model_pfba_solution_select_id.append(eachreaction)

c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['TKT2', 'TPI', 'SUCDi', 'SUCOAS_reverse', 'AKGDH', 'PFK', 'ME2', 'ICL', 'FBA', 'PGI']


Calibration kcat according c13 reaction list

In [17]:
c13reaction_file = './data/C13reaction.csv' 
reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"
f = 0.406
ptot = 0.56 
sigma = 0.5 
enzyme_amount = round(ptot * f * sigma, 3)

c13reaction_selecet=select_calibration_reaction_by_c13(reaction_kcat_mw_file, c13reaction_file, enzyme_amount, 0.1, sigma)
print(c13reaction_selecet)

['GAPD', 'ENO', 'CS', 'AKGDH', 'SUCDi', 'PPC', 'PSERT']


In [18]:
reaction_kcat_mw_file = "./analysis/reaction_kapp_change.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change_c13.csv"
#c13reaction_selecet=['CS','ACONTa','ACONTb','ICDHyr','MALS', 'MDH', 'ICL', 'SUCOAS_reverse', 'SUCDi', 'AKGDH']
kcat_data_colect_file="./analysis/reaction_max_df.csv"
change_reaction_list_round1=change_reaction_kcat_by_database_g(c13reaction_selecet,kcat_data_colect_file,reaction_kcat_mw_file,reaction_kapp_change_file)
print(change_reaction_list_round1)

['ENO', 'CS', 'AKGDH', 'SUCDi', 'PPC', 'PSERT']


In [19]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_round2.json"
f = 0.406
ptot = 0.56 
sigma = 0.5
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model_split_isoenzyme(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [20]:
json_model_path = "./model/iML1515_irr_enz_constraint_adj_round2.json"
fluxes_outfile = './analysis/ECMpy_kapp_adj_round2_solution_df_pfba.csv'
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13.csv"
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile,reaction_kcat_mw_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.45575627423644105


In [21]:
import re
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = []
for eachreaction in enz_model_pfba_solution_select.index:
    if re.search('_num',eachreaction):
        enz_model_pfba_solution_select_id.append(eachreaction.split('_num')[0])
    else:
        enz_model_pfba_solution_select_id.append(eachreaction)

c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['TKT2', 'TPI', 'GND', 'RPE', 'TALA', 'PFK', 'ME2', 'TKT1', 'ICL', 'FBA']


Calibration kcat according Enzyme usage 

In [22]:
f = 0.406
ptot = 0.56 
sigma = 0.5
enz_total = round(ptot * f * sigma, 3)
reaction_flux_file = './analysis/ECMpy_kapp_adj_round2_solution_df_pfba.csv' 
reaction_kcat_mw_file = "./analysis/reaction_kapp_change_c13.csv"
reaction_enz_usage_file = './analysis/ECMpy_kapp_adj_round2_reaction_enz_usage_df.csv'

reaction_enz_usage_df = get_enzyme_usage(enz_total,reaction_flux_file,reaction_kcat_mw_file,reaction_enz_usage_file)
reaction_enz_usage_df.head(22)

,kcat_mw,flux,enz useage,enz ratio
ATPS4rpp_num2,785.767390,29.809919,0.037937,0.332784
PDH_num1,999.074397,15.025193,0.015039,0.131922
PSERT,112.660337,0.758005,0.006728,0.059020
PAPSR2_num2,33.787531,0.112791,0.003338,0.029283
GAPD,2950.918009,9.300138,0.003152,0.027646
KARA2,42.209426,0.132410,0.003137,0.027517
EDA,3885.326983,9.603359,0.002472,0.021682
ACGS,64.093962,0.134809,0.002103,0.018450
SADT2,59.309492,0.112791,0.001902,0.016682
PGL,5091.247502,9.603359,0.001886,0.016546


In [23]:
select_reaction = list(reaction_enz_usage_df[reaction_enz_usage_df['enz ratio']>0.01].index)#more than 1%
print(select_reaction)
reaction_kcat_mw_file = "./analysis/reaction_kapp_change_c13.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change_c13_enzuse.csv"
kcat_data_colect_file="./analysis/reaction_max_df.csv"

change_reaction_list_round1=change_reaction_kcat_by_database_g(select_reaction,kcat_data_colect_file,reaction_kcat_mw_file,reaction_kapp_change_file)
print(change_reaction_list_round1)

['ATPS4rpp_num2', 'PDH_num1', 'PSERT', 'PAPSR2_num2', 'GAPD', 'KARA2', 'EDA', 'ACGS', 'SADT2', 'PGL', 'ADSS', 'CYTBO3_4pp', 'IPPMIb_reverse', 'IPPMIa_reverse', 'ACCOAC']
['ATPS4rpp_num2', 'PAPSR2_num2', 'KARA2', 'SADT2', 'ADSS', 'CYTBO3_4pp', 'IPPMIb_reverse', 'IPPMIa_reverse', 'ACCOAC']


In [24]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13_enzuse.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_round3.json"
f = 0.406
ptot = 0.56 
sigma = 0.5
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model_split_isoenzyme(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [25]:
json_model_path = "./model/iML1515_irr_enz_constraint_adj_round3.json"
fluxes_outfile = './analysis/ECMpy_kapp_adj_round3_solution_df_pfba.csv'
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13_enzuse.csv"
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile,reaction_kcat_mw_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.6994329816264653


In [26]:
import re
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = []
for eachreaction in enz_model_pfba_solution_select.index:
    if re.search('_num',eachreaction):
        enz_model_pfba_solution_select_id.append(eachreaction.split('_num')[0])
    else:
        enz_model_pfba_solution_select_id.append(eachreaction)

c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['TKT2', 'TPI', 'ACKr_reverse', 'PTAr', 'PFK', 'ME2', 'PYK', 'ICL', 'FBA', 'PGI']


# Step4: Solveing enzyme concentration constraint by COBRApy.

In [21]:
#run enzyme constraint metabolic model
json_model_path="./model/iML1515_irr_enz_constraint_adj_round2.json"
enz_model=get_enzyme_constraint_model(json_model_path)
pfba_solution = cobra.flux_analysis.pfba(enz_model)
pfba_solution_df = pfba_solution.to_frame()
#pfba_solution_df.to_csv('./analysis/ECMpy_solution_df_pfba.csv')
pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']

0.7901116510198356

In [22]:
#run genome-scale metabolic model
norm_model=cobra.io.json.load_json_model(json_model_path)
pfba_solution = cobra.flux_analysis.pfba(norm_model)
pfba_solution_df = pfba_solution.to_frame()
#pfba_solution_df.to_csv('./analysis/Orimodel_solution_df_pfba.csv')
pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']


0.8769972144269698